### OOP Object Oriented Programming

Init method we define the model layers

Forward method describes what happens to the input when passed to the model

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch.nn.init as init

import pandas as pd
from torchmetrics import Accuracy

from sklearn.model_selection import train_test_split

In [6]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Define the three linear layers
        self.fc1 = nn.Linear(4,16)
        self.fc2 = nn.Linear(16,8)
        self.fc3 = nn.Linear(8,1)
        
    def forward(self, x):
        # Pass x through linear layers adding activations
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.sigmoid(self.fc3(x))
        return x

In [7]:
df = pd.read_csv("Data/water_potability.csv")
df = df.dropna()

X = df[['ph', 'Sulfate', 'Conductivity', 'Organic_carbon']].values
y = df['Potability'].values



# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32).reshape(-1, 1)


In [8]:
def train_model(optimizer, net, criterion, train_loader, val_loader, num_epochs):
    for epoch in range(num_epochs):
        net.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        print(f'Train Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss}')

        # Validation
        net.eval()
        val_loss = sum(criterion(net(inputs), labels) for inputs, labels in val_loader)
        avg_val_loss = val_loss / len(val_loader)
        print(f'Validation Loss: {avg_val_loss}')





### Comparing different optimizers

Optimizer

Update parameters


SGD (Stochsatic Gradient Descent)
- Update depends on learning rate
- Simple and efficient for basic models 
- Rarely used in practice

Adagrad (Adaptive Gradient)
- Adapts learning rate for each param
- Good for sparse data
- May decrease lr to fast

RMSprop (Root Mean Square Propagation)
- Update lr for each param based on size of its prev gradients

Adam (Adaptive Moment Estimaton)
- Most versatile and used
- RMSprop + momentum


In [9]:


train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

net = Net()
optimizer = optim.SGD(net.parameters(), lr=0.001)
criterion = nn.BCELoss() 

train_model(optimizer, net, criterion, train_loader, val_loader, num_epochs=10)

Train Epoch 1/10, Loss: 1.249720878460828
Validation Loss: 0.7630038857460022
Train Epoch 2/10, Loss: 0.7356672029869229
Validation Loss: 0.6685841083526611
Train Epoch 3/10, Loss: 0.7105151286312178
Validation Loss: 0.6981305480003357
Train Epoch 4/10, Loss: 0.7038150441412833
Validation Loss: 0.6801831722259521
Train Epoch 5/10, Loss: 0.7154992900642694
Validation Loss: 0.94149249792099
Train Epoch 6/10, Loss: 0.6961218615372976
Validation Loss: 0.6849472522735596
Train Epoch 7/10, Loss: 0.6921929646940792
Validation Loss: 0.676598846912384
Train Epoch 8/10, Loss: 0.6966932451023775
Validation Loss: 0.7226216197013855
Train Epoch 9/10, Loss: 0.6896803086879206
Validation Loss: 0.796906054019928
Train Epoch 10/10, Loss: 0.6831797863922867
Validation Loss: 0.6870715022087097


In [10]:
optimizer = optim.RMSprop(net.parameters(), lr=0.001)
criterion = nn.BCELoss() 

train_model(optimizer, net, criterion, train_loader, val_loader, num_epochs=10)

Train Epoch 1/10, Loss: 0.6989837300543692
Validation Loss: 0.6801596283912659
Train Epoch 2/10, Loss: 0.675970221267027
Validation Loss: 0.70899498462677
Train Epoch 3/10, Loss: 0.6816726467188667
Validation Loss: 0.713750422000885
Train Epoch 4/10, Loss: 0.6761585953188878
Validation Loss: 0.6883933544158936
Train Epoch 5/10, Loss: 0.6743184257956112
Validation Loss: 0.6841436624526978
Train Epoch 6/10, Loss: 0.6760777363590166
Validation Loss: 0.6792064309120178
Train Epoch 7/10, Loss: 0.6757895362143423
Validation Loss: 0.6807308793067932
Train Epoch 8/10, Loss: 0.6690469068639419
Validation Loss: 0.7493492960929871
Train Epoch 9/10, Loss: 0.6756961076867347
Validation Loss: 0.6940949559211731
Train Epoch 10/10, Loss: 0.675023642240786
Validation Loss: 0.6920475363731384


In [11]:
optimizer = optim.Adam(net.parameters(),lr=0.001)

criterion = nn.BCELoss() 

train_model(optimizer, net, criterion, train_loader, val_loader, num_epochs=10)

Train Epoch 1/10, Loss: 0.6768721038219976
Validation Loss: 0.6799663305282593
Train Epoch 2/10, Loss: 0.6736545036820805
Validation Loss: 0.6785370707511902
Train Epoch 3/10, Loss: 0.670043640276965
Validation Loss: 0.6820958852767944
Train Epoch 4/10, Loss: 0.6758833586000929
Validation Loss: 0.6835234761238098
Train Epoch 5/10, Loss: 0.6701976832221536
Validation Loss: 0.686634361743927
Train Epoch 6/10, Loss: 0.6722953611729192
Validation Loss: 0.6756556630134583
Train Epoch 7/10, Loss: 0.6705967421625175
Validation Loss: 0.6738203763961792
Train Epoch 8/10, Loss: 0.6677833100159963
Validation Loss: 0.6864558458328247
Train Epoch 9/10, Loss: 0.6739162837757784
Validation Loss: 0.6744876503944397
Train Epoch 10/10, Loss: 0.667724259928161
Validation Loss: 0.6790217161178589


In [14]:
#Check the model's performance 
acc = Accuracy(task="binary")

net.eval()
with torch.no_grad():
    for features, labels in val_loader:
        outputs = net(features)
        preds = (outputs >= 0.5).float()
        acc(preds, labels.view(-1, 1))

test_accuracy = acc.compute()
print(f"Test accuracy: {test_accuracy}")

Test accuracy: 0.5806451439857483


## Unstable gradients
- Vanishing gradients
- Exploding gradients

Solution
- Proper weights
- Good activations
- Batch normalization

In [17]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Define the three linear layers
        self.fc1 = nn.Linear(4,16)
        self.fc2 = nn.Linear(16,8)
        self.fc3 = nn.Linear(8,1)

        # Apply He initialization
        init.kaiming_uniform_(self.fc1.weight)
        init.kaiming_uniform_(self.fc2.weight)
        init.kaiming_uniform_(self.fc3.weight, nonlinearity="sigmoid")

        # Add two batch normalization layers
        self.bn1 = nn.BatchNorm1d(16)
        self.bn2 = nn.BatchNorm1d(8)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = nn.functional.elu(x)

        # Pass x through the second set of layers
        x = self.fc2(x)
        x = self.bn2(x)
        x = nn.functional.elu(x)

        x = nn.functional.sigmoid(self.fc3(x))
        return x

In [18]:
optimizer = optim.RMSprop(net.parameters(), lr=0.001)
criterion = nn.BCELoss() 

train_model(optimizer, net, criterion, train_loader, val_loader, num_epochs=10)

Train Epoch 1/10, Loss: 0.6804442604382833
Validation Loss: 0.6899144649505615
Train Epoch 2/10, Loss: 0.672169002832151
Validation Loss: 0.6919499039649963
Train Epoch 3/10, Loss: 0.6742618247574451
Validation Loss: 0.6803264617919922
Train Epoch 4/10, Loss: 0.6711124240183363
Validation Loss: 0.6828188300132751
Train Epoch 5/10, Loss: 0.6714974419743407
Validation Loss: 0.6745815873146057
Train Epoch 6/10, Loss: 0.672397888174244
Validation Loss: 0.6823580265045166
Train Epoch 7/10, Loss: 0.6710366980702269
Validation Loss: 0.677545964717865
Train Epoch 8/10, Loss: 0.6716990424137489
Validation Loss: 0.6895720958709717
Train Epoch 9/10, Loss: 0.6711431215791142
Validation Loss: 0.6781908869743347
Train Epoch 10/10, Loss: 0.6703510810347164
Validation Loss: 0.6801164150238037


In [21]:
#Check the model's performance
acc = Accuracy(task="binary")

net.eval()
with torch.no_grad():
    for features, labels in val_loader:
        outputs = net(features)
        preds = (outputs >= 0.5).float()
        acc(preds, labels.view(-1, 1))

test_accuracy = acc.compute()
print(f"Test accuracy: {test_accuracy}")

Test accuracy: 0.578163743019104
